In [1]:
"""
    Скрипт читает ссылки на rtsp-потоки из файла
    Создаёт список уникальных ссылок
    Создаёт экземпляр cv2.VideoCapture для каждой ссылки
    Создаёт батч из всех фреймов полученных из всех потоков
    Отправляет батч в модель
    Сохраняет все фреймы где есть файлы с нужными классами в отдельную папку
    Вместе с фреймами сохраняются боундинг боксы и классы обнаруженных объектов
"""
import logging
import cv2
from multiprocessing import Process, Queue
from urllib.parse import urlparse
from time import sleep

logging.basicConfig(level=logging.DEBUG)

In [2]:
def is_url(s):
    try:
        result = urlparse(s)
        return all((result.scheme, result.netloc))
    except ValueError:
        return False
    
class VideoReader(Process):
    def __init__(self, link, frame_queue):
        super().__init__()
        self.is_valid = is_url(link)
        self.link = link
        self.frame_queue = frame_queue
        self.cap = None
        logging.debug(f'{self.name} created with link {link}')
        
    def run(self):
        while True:
            if not self.is_cap_open():
                self.cap = cv2.VideoCapture(self.link)
                logging.debug(f'{self.name} opened {self.link}')
                sleep(1)
            ret, frame = self.cap.read()
            if not ret:
                self.cap.release()
                self.cap = None
                logging.debug(f'{self.name} closed {self.link}')
                sleep(1)
                continue
            # preprocessing here
            logging.debug(f'{self.name} put frame from {self.link} to queue')
            self.frame_queue.put((self.name, frame))
            
    def stop(self):
        if self.is_cap_open():
            self.cap.release()
            logging.debug(f'{self.name} terminated {self.link}')
        self.terminate()
        
    def is_cap_open(self):
        return self.cap is not None and self.cap.isOpened()

In [3]:
# Читаем уникальные ссылки из файла
data_file = 'data/rtsp_links.txt'
links = []
with open(data_file, 'r') as file:
    links = file.readlines()[:5]
links = list(set(links))
print(f'Found {len(links)} unique links')

Found 5 unique links


In [4]:
frame_queue = Queue()
readers = [VideoReader(link, frame_queue) for link in links]

for p in readers:
    p.start()
    
k = 0

while k < 20:
    if not frame_queue.empty():
        k+=1
        process_name, frame = frame_queue.get()
        print(f'Получен {process_name}')
        # Обработка здесь

DEBUG:root:VideoReader-1 created with link https://live.cmirit.ru:443/live/ever1_1920x1080.stream/playlist.m3u8

DEBUG:root:VideoReader-2 created with link https://live.cmirit.ru:443/live/fotomost01_1920x1080.stream/playlist.m3u8

DEBUG:root:VideoReader-3 created with link https://live.cmirit.ru:443/live/arh-most-002.stream/playlist.m3u8

DEBUG:root:VideoReader-4 created with link https://live.cmirit.ru:443/live/arh-most-001.stream/playlist.m3u8

DEBUG:root:VideoReader-5 created with link https://live.cmirit.ru:443/live/a-alleya-03_1920x1080.stream/playlist.m3u8



KeyboardInterrupt: 